In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import base64
from openai import OpenAI
import fitz  # PyMuPDF
from pathlib import Path

In [5]:
downloads = Path("./../downloads")
pdfs = list((downloads / "pdfs").iterdir())
images = downloads / "images"
dishes = [p.name.replace(".pdf", "") for p in pdfs]

In [6]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def pdf2image(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    image_pth = images / pdf_path.name.replace(".pdf", "")
    image_pth.mkdir()
    # Iterate through each page and save it as an image
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        pix = page.get_pixmap()
        image_path = f'page_{page_num + 1}.png'
        pix.save(image_path)
        encode_image(image_path)

    pdf_document.close()

In [7]:
_ = [pdf2image(p) for p in pdfs]

In [ ]:

client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What are in these images? Is there any difference between them?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)